## Profit Based on Final Stock Price

In [1]:
# BTCC-B.TO Call Option Price Evaluation
# All options expire on January 16th 2026

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pytz
from scipy.stats import norm

# binomial_tree_option_priceModel
from scipy.stats import norm
import numpy as np

# Retrieve historical data
ticker = "BTC-CAD"
start_time = (datetime.now(pytz.timezone('US/Pacific')) - timedelta(days=365*4)).strftime('%Y-%m-%d')
end_time = (datetime.now(pytz.timezone('US/Pacific'))).strftime('%Y-%m-%d')
data = yf.download(ticker, start=start_time, end=end_time, interval="1d")[['Close']]
data['Daily_Return'] = data['Close'].pct_change()
daily_volatility = data['Daily_Return'].std()
annualized_volatility = daily_volatility * np.sqrt(252)

initial_capital = 10_000
current_stock_price = 12.50
final_stock_price = 40.00

# List of option call scenarios
# Format: (Strike, Ask Price)
options = [
    (20.00, 2.90),
    (19.00, 2.90),
    (18.00, 3.10),
    (17.00, 3.20),
    (16.00, 3.40),
    (15.00, 3.30),
    (14.00, 3.90),
    (13.00, 4.10),
    (12.00, 4.25),
    (10.00, 4.85),
    (9.00, 5.20),
    (8.00, 5.65),
    (7.00, 6.25),
    (6.00, 6.90),
    (5.00, 8.10),
    (4.00, 8.40),
    (3.00, 9.80)
]

# Function to calculate profit for buying and holding the stock
def calculate_stock_profit(initial_capital, current_price, final_price):
    num_shares = initial_capital // current_price
    return num_shares * (final_price - current_price)

# Function to calculate profit for each option scenario
def calculate_option_profit(initial_capital, strike, ask_price, final_price):
    num_options = initial_capital // (ask_price * 10)
    profit_per_option = max(0, final_price - strike) - ask_price
    return num_options * profit_per_option * 10

# Calculate profit for buying and holding
stock_profit = calculate_stock_profit(initial_capital, current_stock_price, final_stock_price)
print(f"Profit from buying and holding the stock: ${stock_profit:.2f}")

# Calculate and display profit for each option scenario
for strike, ask_price in options:
    option_profit = calculate_option_profit(initial_capital, strike, ask_price, final_stock_price)
    print(f"Profit from option with strike ${strike} and ask price ${ask_price}: ${option_profit:.2f}")


[*********************100%%**********************]  1 of 1 completed

Profit from buying and holding the stock: $22000.00
Profit from option with strike $20.0 and ask price $2.9: $58824.00
Profit from option with strike $19.0 and ask price $2.9: $62264.00
Profit from option with strike $18.0 and ask price $3.1: $60858.00
Profit from option with strike $17.0 and ask price $3.2: $61776.00
Profit from option with strike $16.0 and ask price $3.4: $60564.00
Profit from option with strike $15.0 and ask price $3.3: $65751.00
Profit from option with strike $14.0 and ask price $3.9: $56576.00
Profit from option with strike $13.0 and ask price $4.1: $55647.00
Profit from option with strike $12.0 and ask price $4.25: $55812.50
Profit from option with strike $10.0 and ask price $4.85: $51809.00
Profit from option with strike $9.0 and ask price $5.2: $49536.00
Profit from option with strike $8.0 and ask price $5.65: $46376.00
Profit from option with strike $7.0 and ask price $6.25: $42800.00
Profit from option with strike $6.0 and ask price $6.9: $39024.00
Profit fro


C:\Users\falty\AppData\Local\Temp\ipykernel_18576\1473846985.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Daily_Return'] = data['Close'].pct_change()


## Binomial Tree

In [2]:
import numpy as np
from scipy.stats import skew, kurtosis

# Assuming data['Daily_Return'] contains the daily returns
historical_volatility = np.std(np.log(1 + data['Daily_Return'].dropna())) * np.sqrt(252)  # Annualized
mean_return = np.mean(data['Daily_Return'].dropna()) * 252  # Annualized

historical_skewness = skew(data['Daily_Return'].dropna())
historical_kurtosis = kurtosis(data['Daily_Return'].dropna(), fisher=False)

def binomial_tree_option_price(S, K, T, r, sigma, N=20_000, dividends=0, skew=0, kurtosis=0, drift=0):
    dt = T / N
    u = np.exp((drift - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt))  # Incorporating drift
    d = 1 / u
    p = (np.exp((r - dividends) * dt) - d) / (u - d)

    p += skew * 0.001 * (1 - 2 * p) + kurtosis * 0.0005 * (1 - 2 * p)
    p = min(max(p, 0), 1)

    prices = S * d**np.arange(N, -1, -1) * u**np.arange(0, N+1, 1)
    option_values = np.maximum(prices - K, 0)

    discount_factor = np.exp(-r * dt)
    for i in range(N - 1, -1, -1):
        option_values[:i+1] = (p * option_values[1:i+2] + (1 - p) * option_values[:i+1]) * discount_factor
        # For American options, compare with early exercise
        option_values[:i+1] = np.maximum(option_values[:i+1], prices[:i+1] - K)

    return option_values[0]

# Example usage:
risk_free_rate = 0.05  # Example rate
T = 2  # years until expiration
N = 100  # number of steps

# Example usage
for strike, ask_price in options:
    option_price = binomial_tree_option_price(
        current_stock_price, strike, T, risk_free_rate, historical_volatility, N,
        dividends=0, skew=historical_skewness, kurtosis=historical_kurtosis, drift=mean_return
    )
    print(f"Call Option Price for strike {strike}: ${option_price:.2f}")

Call Option Price for strike 20.0: $2.67
Call Option Price for strike 19.0: $2.86
Call Option Price for strike 18.0: $3.05
Call Option Price for strike 17.0: $3.28
Call Option Price for strike 16.0: $3.52
Call Option Price for strike 15.0: $3.77
Call Option Price for strike 14.0: $4.07
Call Option Price for strike 13.0: $4.38
Call Option Price for strike 12.0: $4.73
Call Option Price for strike 10.0: $5.53
Call Option Price for strike 9.0: $5.98
Call Option Price for strike 8.0: $6.50
Call Option Price for strike 7.0: $7.06
Call Option Price for strike 6.0: $7.68
Call Option Price for strike 5.0: $8.35
Call Option Price for strike 4.0: $9.09
Call Option Price for strike 3.0: $9.89


## Monte-Carlo Simulation 

In [3]:
import numpy as np
from scipy.stats import norm
from numba import jit  # Import numba for JIT compilation

In [4]:
%%time
from numba import jit, prange
import numpy as np

@jit(nopython=True, parallel=True)  # Enable JIT compilation with parallel execution
def monte_carlo_option_price(S, K, T_months, r, sigma, lambda_, mu_J, sigma_J, num_paths=10000, num_steps=100, skew=0, kurtosis=0):
    """
    Monte Carlo simulation for European call option pricing with jump diffusion.

    Parameters:
    S (float): Current stock price.
    K (float): Strike price of the option.
    T_months (int): Time to maturity in months.
    r (float): Risk-free interest rate.
    sigma (float): Volatility of the underlying asset.
    lambda_ (float): Jump intensity, representing the frequency of jumps.
    mu_J (float): Mean of the logarithm of the jump size.
    sigma_J (float): Standard deviation of the logarithm of the jump size.
    num_paths (int): Number of simulated paths in the Monte Carlo simulation.
    num_steps (int): Number of time steps in each simulated path.
    skew (float): Skewness of the asset returns. Defaults to 0 for no skew.
    kurtosis (float): Kurtosis of the asset returns. Defaults to 0 for normal kurtosis.

    Returns:
    float: Estimated price of the European call option.
    """
    T = T_months / 12  # Convert maturity from months to years
    dt = T / num_steps  # Time step for simulation
    discount_factor = np.exp(-r * T)  # Discount factor for present value

    payoffs = np.zeros(num_paths)  # Initialize array for option payoffs
    for i in prange(num_paths):  # Parallel loop for each path
        S_t = S  # Initial stock price for this path
        for j in range(num_steps):  # Time step loop
            z = np.random.normal()  # Standard normal random variable
            shock = z + skew * (z ** 2 - 1)  # Adjust random variable for skew and kurtosis
            num_jumps = np.random.poisson(lambda_ * dt)  # Poisson process for number of jumps
            jump_sum = np.sum(np.random.lognormal(mu_J, sigma_J, num_jumps) - 1)  # Sum of jump sizes
            S_t *= np.exp((r - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * shock + jump_sum)  # Stock price update
        payoffs[i] = max(S_t - K, 0)  # Call option payoff for this path

    return discount_factor * np.mean(payoffs)  # Average discounted payoff

# Example usage
for strike, ask_price in options:
    T_months = 23  # Option maturity in months
    lambda_ = 0.2  # Jump intensity
    mu_J = 0.1  # Mean of log-normal jump size
    sigma_J = 0.1  # Standard deviation of log-normal jump size
    option_price = monte_carlo_option_price(
        current_stock_price, strike, T_months, risk_free_rate, historical_volatility,
        lambda_, mu_J, sigma_J,
        num_paths=100_000, num_steps=T_months*730,  # Adjusted number of paths and steps
        skew=historical_skewness, kurtosis=historical_kurtosis
    )
    print(f"Monte Carlo Call Option Price for strike {strike}: ${option_price:.2f}")

Monte Carlo Call Option Price for strike 20.0: $5.87
Monte Carlo Call Option Price for strike 19.0: $6.17
Monte Carlo Call Option Price for strike 18.0: $6.38
Monte Carlo Call Option Price for strike 17.0: $6.81
Monte Carlo Call Option Price for strike 16.0: $6.98
Monte Carlo Call Option Price for strike 15.0: $7.29
Monte Carlo Call Option Price for strike 14.0: $7.69
Monte Carlo Call Option Price for strike 13.0: $8.02
Monte Carlo Call Option Price for strike 12.0: $8.48
Monte Carlo Call Option Price for strike 10.0: $9.37
Monte Carlo Call Option Price for strike 9.0: $9.84
Monte Carlo Call Option Price for strike 8.0: $10.42
Monte Carlo Call Option Price for strike 7.0: $10.89
Monte Carlo Call Option Price for strike 6.0: $11.73
Monte Carlo Call Option Price for strike 5.0: $12.25
Monte Carlo Call Option Price for strike 4.0: $13.01
Monte Carlo Call Option Price for strike 3.0: $13.80
CPU times: total: 25min 36s
Wall time: 57.1 s
